# Practice on Keras Functional API

In [1]:
# imports
import keras
from keras import layers
from keras import backend as K
from keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# for types hints
from typing import Tuple, Callable, List
from tensorflow import Tensor

In [2]:
# mnist dataset
def load_and_preprocess_mnist() -> Tuple[Tuple[Tensor, Tensor], Tuple[Tensor, Tensor]]:
    """
    Load the MNIST dataset, preprocess images, and perform one-hot encoding of labels.

    :return: Tuple of training data (x_train, y_train) and testing data (x_test, y_test).
    """
    # Load MNIST dataset
    (x_train, y_train), (x_test, y_test) =  keras.datasets.mnist.load_data()

    # Reshape and normalize images
    x_train = x_train.reshape(60000, 784).astype('float32') / 255
    x_test = x_test.reshape(10000, 784).astype('float32') / 255

    # One-hot encoding of labels
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    return (x_train, y_train), (x_test, y_test)

In [3]:

(x_train, y_train), (x_test, y_test) = load_and_preprocess_mnist()


In [4]:
# building the model
def build_model() -> keras.Model:
    """
    Build a simple MLP model for MNIST classification.
    :return: A Keras Model with inputs and outputs defined.
    """
    inputs = keras.Input(shape=(784,), name='input')  # input layer
    x = layers.Dense(64, activation='relu', name='hidden1')(inputs)  # hidden layer
    x = layers.Dense(64, activation='relu', name='hidden2')(x)  # hidden layer
    outputs = layers.Dense(10, activation='softmax', name='output')(x)  # output layer
    
    return keras.Model(inputs=inputs, outputs=outputs)

model_ce = build_model()
model_fl = build_model()
model_rl = build_model()

In [5]:
optimizer = keras.optimizers.SGD()
ce_loss = keras.losses.CategoricalCrossentropy()
metrics = [keras.metrics.CategoricalAccuracy()]
# compiling the model
model_ce.compile(optimizer=optimizer, loss=ce_loss, metrics=metrics)
# training the model
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_ce.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2, callbacks=[early_stopping_callback])
# evaluating the model
test_scores = model_ce.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

# saving the model
path = './weights/mnist_ce_model.keras'
model_ce.save(path)

Epoch 1/100
750/750 [==============================] - 2s 2ms/step - loss: 1.1003 - categorical_accuracy: 0.7094 - val_loss: 0.5039 - val_categorical_accuracy: 0.8724
Epoch 2/100
750/750 [==============================] - 1s 2ms/step - loss: 0.4460 - categorical_accuracy: 0.8769 - val_loss: 0.3618 - val_categorical_accuracy: 0.8986
Epoch 3/100
750/750 [==============================] - 1s 2ms/step - loss: 0.3588 - categorical_accuracy: 0.8980 - val_loss: 0.3167 - val_categorical_accuracy: 0.9090
Epoch 4/100
750/750 [==============================] - 1s 2ms/step - loss: 0.3200 - categorical_accuracy: 0.9081 - val_loss: 0.2891 - val_categorical_accuracy: 0.9166
Epoch 5/100
750/750 [==============================] - 1s 2ms/step - loss: 0.2933 - categorical_accuracy: 0.9159 - val_loss: 0.2716 - val_categorical_accuracy: 0.9217
Epoch 6/100
750/750 [==============================] - 2s 3ms/step - loss: 0.2727 - categorical_accuracy: 0.9221 - val_loss: 0.2522 - val_categorical_accuracy: 0.927

KeyboardInterrupt: 

In [ ]:
focal_loss = keras.losses.CategoricalFocalCrossentropy()
# compiling the model
optimizer = keras.optimizers.SGD()
model_fl.compile(optimizer=optimizer, loss=focal_loss, metrics=metrics)
# training the model
model_fl.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2, callbacks=[early_stopping_callback])
# evaluating the model
test_scores = model_fl.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])
# saving the model
path = './weights/mnist_focal_model.keras'
model_fl.save(path)

In [6]:
# defining rational loss function
# RL(p_t) = 1/p_t * -p * log(p_t)
    # TODO: plot the function
def rational_loss(y_true, y_pred):
    """
    Rational Loss for multi-class classification, Keras style.
    RL(p_t) = - 1/p_t * log(p_t), where p_t is the probability associated with the true class.

    :param y_true: Ground truth labels, shape of [batch_size, num_classes].
    :param y_pred: Predicted class probabilities, shape of [batch_size, num_classes].
    :return: A scalar representing the mean rational loss over the batch.
    NOTE: written assuming GPU support to make use of fast Tensor operations.
    """
    # Create a Categorical Cross-Entropy loss instance
    cce = keras.losses.CategoricalCrossentropy(
        reduction=keras.losses.Reduction.NONE # Keep unreduced loss tensor
    )
    # Clip the prediction value to prevent NaN's and Inf's
    _y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
    cross_entropy = cce(y_true, _y_pred) # batch_sizex1
    # get y_pred for the true class
    _y_pred = K.sum(y_true * _y_pred, axis=1) # batch_sizex1
    _y_pred = K.clip(_y_pred, K.epsilon(), None) # Avoid division by zero
    _rational_loss = cross_entropy / _y_pred # Rational loss  
    
    return _rational_loss


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [7]:
# test rational loss
y_true = np.array([[0, 1, 0], [0, 0, 1]])
y_pred = np.array([[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
print(rational_loss(y_true, y_pred))



tf.Tensor([ 0.05399305 23.02585093], shape=(2,), dtype=float64)


In [ ]:
# def generate_data() -> Tuple[np.ndarray, np.ndarray]:
#     """
#     Generates synthetic y_true and y_pred data.
#     :return: y_true and y_pred arrays.
#     """
#     num_samples = 100
#     num_classes = 3
#     y_true = np.eye(num_classes)[np.random.choice(num_classes, num_samples)]
#     y_pred = np.random.rand(num_samples, num_classes)
#     y_pred /= y_pred.sum(axis=1, keepdims=True)
#     return y_true, y_pred
# 
# def plot_rational_loss() -> None:
#     """
#     Plots the rational loss for the generated data.
#     """
#     y_true, y_pred = generate_data()
#     rational_loss_fixed = rational_loss()
#     losses: List[float] = [rational_loss_fixed(y_t.reshape(1, -1), y_p.reshape(1, -1)).numpy() for y_t, y_p in zip(y_true, y_pred)]
#     plt.plot(losses)
#     plt.title("Rational Loss for Multi-Class Classification")
#     plt.xlabel("Sample")
#     plt.ylabel("Loss")
#     plt.show()
# 
# plot_rational_loss()

In [ ]:
# compiling the model
model_rl.compile(optimizer= keras.optimizers.SGD(), loss=rational_loss, metrics=metrics)
# training the model
history = model_rl.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2)
# evaluating the model
test_scores = model_rl.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])
# saving the model
path = './weights/mnist_rational_model.keras'
model_rl.save(path)

In [ ]:
# rebuilding the models
model_ce = build_model()
model_fl = build_model()
model_rl = build_model()


In [ ]:
# imbalance 
def create_imbalanced_data(x, y, imbalance_rate=0.5):
    """
    Create an imbalanced dataset based on a given probability distribution.
    The probability for class d is given by: P(d) = 0.5^d / 2*(1 - 0.5^10)

    :param x: Features, shape of [total_samples, feature_dim].
    :param y: One-hot encoded labels, shape of [total_samples, num_classes].
    :param imbalance_rate: Base rate for the exponential decay of class frequency (default 0.5).
    :return: Tuple of imbalanced features and labels, shapes of [selected_samples, feature_dim] and [selected_samples, num_classes].
    """
    total_samples = len(y)
    a = imbalance_rate
    normalization_factor = 2 * (1 - a**10)

    indices_by_class = [np.where(y[:, d] == 1)[0] for d in range(10)]
    selected_indices = []

    for d in range(10):
        probability_d = (a**d) / normalization_factor
        frequency = int(total_samples * probability_d)
        np.random.shuffle(indices_by_class[d]) # Shuffle to ensure random selection
        selected_indices.extend(indices_by_class[d][:frequency])

    return x[selected_indices], y[selected_indices]


x_train_imbalanced, y_train_imbalanced = create_imbalanced_data(x_train, y_train)


In [ ]:
# compiling the model
model_ce.compile(optimizer=keras.optimizers.SGD(), loss=keras.losses.CategoricalCrossentropy(), metrics=metrics)
model_fl.compile(optimizer=keras.optimizers.SGD(), loss=keras.losses.CategoricalFocalCrossentropy(), metrics=metrics)
model_rl.compile(optimizer=keras.optimizers.SGD(), loss=rational_loss, metrics=metrics)

In [ ]:
print("Training on imbalanced data:")
print("Cross Entropy:")
model_ce.fit(x_train_imbalanced, y_train_imbalanced, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping_callback])
print("Focal Loss:")
model_fl.fit(x_train_imbalanced, y_train_imbalanced, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping_callback])
print("Rational Loss:")
model_rl.fit(x_train_imbalanced, y_train_imbalanced, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping_callback])

In [ ]:
# Balanced data
print(model_ce.evaluate(x_test, y_test))
print(model_fl.evaluate(x_test, y_test))
print(model_rl.evaluate(x_test, y_test))

# Imbalanced data
x_test_imbalanced, y_test_imbalanced = create_imbalanced_data(x_test, y_test)
print(model_ce.evaluate(x_test_imbalanced, y_test_imbalanced))
print(model_fl.evaluate(x_test_imbalanced, y_test_imbalanced))
print(model_rl.evaluate(x_test_imbalanced, y_test_imbalanced))

In [ ]:
def accuracy_by_bins(model, x, y):
    """
    Calculate and print the accuracy of the given model for specific bins of classes.
    The bins are defined as: 0-1, 2-7, 8-9.

    :param model: Trained Keras model to evaluate.
    :param x: Input features, shape of [num_samples, feature_dim].
    :param y: One-hot encoded labels, shape of [num_samples, num_classes].
    """
    predictions = model.predict(x).argmax(axis=-1)
    true_labels = y.argmax(axis=-1)
    bins = [(0, 1), (2, 7), (8, 9)]
    for bin_start, bin_end in bins:
        mask = (true_labels >= bin_start) & (true_labels <= bin_end) 
        bin_accuracy = np.mean(predictions[mask] == true_labels[mask])
        print(f"Accuracy for bin {bin_start}-{bin_end}: {bin_accuracy}")

print("Accuracy by bins for balanced data:")
print("Cross Entropy:")
accuracy_by_bins(model_ce, x_test, y_test)
print("Focal Loss:")
accuracy_by_bins(model_fl, x_test, y_test)
print("Rational Loss:")
accuracy_by_bins(model_rl, x_test, y_test)